In [1]:
import pandas as pd
import numpy as np
import sys, os, shutil, gc

In [ ]:
# ''' Version of the packages in this work.'''
# # print("Pandas version: ",pd.__version__)
# print("Pandas version: 0.25.1")
# # print("Numpy version: ",np.__version__)
# print("Numpy version: 1.16.4")
# # print("Sys python version: ",sys.version)
# print("Sys python version: 3.7.1 | package by conda-forge [MSC v.1900 64 bit (AMD64)]")
# print("IPython: 7.8.0")
# print("IPython genutils: 0.2.0")

###  Header

In [38]:
#toy example
# input_data_pd = pd.read_csv("./data/toy_example/input_test2.dat", header=None, names=["transation"])

#synthetic
# path = "./data/synthetic/fimi/"
# syn_datasets = [path+"synthetic-1_fimi.dat",path+"synthetic-2_fimi.dat",path+"synthetic-3_fimi.dat"]

experiment = 'Real' # Syn: run the synthetic data analysis; Real: run the real data analysis; 
                   # Toy: run the toy example.
find_overlap = True # True: find overlapped co-clusters; False: Find no overlapped co-clusters
VERBOSE = False # True: print results as a verbose mode; False: print the main results
num_of_sim = 1 # number of simulations to perform
k = -1 #max number of co-cluster that could be found. -1: default [driven by cost function]
e_obj = 0 # maximum error tolerance for object. -1: default [accept the maximum error]
e_att = 0 # maximum error tolerance for attribute. -1: default [accept the maximum error]


print("Executing OCoClus method")
if experiment == 'Syn':
    path = "./data/synthetic/fimi/"
    syn_datasets = [path+"synthetic-1_fimi.dat",path+"synthetic-2_fimi.dat",path+"synthetic-3_fimi.dat"]
    path_method = "OutputAnalysis\ococlus"
    check_path(path_method)

    for ds in range(len(syn_datasets)):
        ds_name = "Syn-"+str(ds+1)
        print("\nDataset: "+ds_name)
        res = os.mkdir(path_method+"\\"+ds_name)

        for run in range(num_of_sim):
            print("Run-"+str(run+1))

            df_fimi = pd.read_csv(syn_datasets[ds], header=None, names=["transation"])
            D,co_clusters = OCoClus(df_fimi,k,e_obj,e_att) # Calling OCoClus main method

            print("")
            Rec_error(D,co_clusters)
#             print(co_clusters)
            
            omega_format = build_clustering_output_omega(co_clusters)
            OCoClus_clustering_xm = xmeasures_format(omega_format)# save to XMEASURES format C++ version
            df_gt = pd.DataFrame(OCoClus_clustering_xm)
            path = path_method+"/"+ds_name
            df_gt.to_csv(path.replace("\\","/")+"/run_"+str(run+1)+"_res_ococlus_"+ds_name+"_co.cnl", 
                         header= False,index=False, encoding='utf8')
            del omega_format, df_gt, OCoClus_clustering_xm
            gc.collect()
elif experiment == 'Real':
    k = 10
    print('Real data clustering.')
    path = "./data/real_application/"
    real_datasets = [path+"cal500_fimi.dat",path+"covid19_fimi.dat"]
    path_method = "OutputAnalysis\ococlus"
    check_path(path_method)
    
    for ds in range(len(real_datasets)):
        ds_name = real_datasets[ds].replace("/","_").split("_")[4].capitalize()
        print("\nDataset: "+ds_name)
        res = os.mkdir(path_method+"\\"+ds_name)
        
        df_fimi = pd.read_csv(real_datasets[ds], header=None, names=["transation"])
        D,co_clusters = OCoClus(df_fimi,k,e_obj,e_att) # Calling OCoClus main method
        writeFileOutput(co_clusters,ds_name,method='OCoClus',fileName='OCoClusResult_'+ds_name)
        
    print('DONE!')
elif experiment == 'Toy':
    print('Toy example')
    input_data_pd = pd.read_csv("./data/toy_example/input_test2.dat", header=None, names=["transation"])
    D,co_clusters = OCoClus(input_data_pd,k,e_obj,e_att)
    #Compute the measures
#     print("")
    Rec_error(D,co_clusters)
else:
    print('ERROR! Choose a valid option for the experiment analysis.')

Executing OCoClus method
Real data clustering.
C:\Users\yurin\Documents\my_github\ococlus
OCoClus' folder was cleaned.

Dataset: Cal500
Iterator:  1
Find dense cocluster method.
Expand co-cluster method.

Iterator:  2
Find dense cocluster method.
Expand co-cluster method.

Iterator:  3
Find dense cocluster method.
Expand co-cluster method.

Iterator:  4
Find dense cocluster method.
Expand co-cluster method.

Iterator:  5
Find dense cocluster method.
Expand co-cluster method.

Iterator:  6
Find dense cocluster method.
Expand co-cluster method.

Iterator:  7
Find dense cocluster method.
Expand co-cluster method.

Iterator:  8
Find dense cocluster method.
Expand co-cluster method.

Iterator:  9
Find dense cocluster method.
Expand co-cluster method.

Iterator:  10
Find dense cocluster method.
Expand co-cluster method.

Find global co-cluster is done.

Find overlapped co-clusters method

Non-overlapped and overlapped co-clusters.
Number of co-clusters:  90
Output file saved in: ./OutputAnal

## Evaluation (Omega index, overlapped F1, ONMI)

In [63]:
%%bash
#!/bin/bash
### omega index and f-score
### ./xmeasures -o -fp -ku -O gt.txt cls2.txt
# pwd
# ls

if [ -d "xmeasures/OutputAnalysis/ococlus" ] 
then
#     echo "Directory exists."
    rm -R xmeasures/OutputAnalysis/ococlus
else
    echo "Error: Directory does not exists."
fi

cp -R OutputAnalysis xmeasures
cd xmeasures/

#Method: [Ococlus]
#ground-truth: [gt_xm_s1_co.cnl,gt_xm_s2_co.cnl,gt_xm_s3_co.cnl]
for i in 1 2 3 # index of synthetic datasets
# for i in 1
do
#     for file in OutputAnalysis/ococlus/Syn-${i}/*
    for file in OutputAnalysis/ococlus/Syn-${i}/*
    do
    #     echo " $(grep -c '' ${file})"
        res=$(grep -c '' ${file})
    #     echo "${file} lines equal to: ${res}"

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
    #         echo "Empty file"
            ./xmeasures -o -fp -ku -O ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/ococlus/Syn-1/run_1_res_ococlus_Syn-1_co.cnl is not empty. It has 7 lines.
 
 
= Overlaps Evaluation =
MF1p_u (PARTPROB, UNWEIGHTED):
1 (Prc: 1, Rec: 1)
OI:
1
MF1p_u: 1 (Prc: 1, Rec: 1); OI: 1


### ONMI

In [62]:
%%bash
#!/bin/bash
#### ONMI
#### ./onmi file1 file2

cd xmeasures

#Method: [Ococlus]
#ground-truth: [gt_xm_s1_trad.cnl,gt_xm_s2_trad.cnl,gt_xm_s3_trad.cnl]
for i in 1 2 3 # index of synthetic datasets
# for i in 1
do
    for file in OutputAnalysis/ococlus/Syn-${i}/*
    do
    #     echo " $(grep -c '' ${file})"
        res=$(grep -c '' ${file})
    #     echo "${file} lines equal to: ${res}"

        if [ ${res} != 0 ]
        then 
            echo " "
            echo "File ${file} is not empty. It has ${res} lines."
    #         echo "Empty file"
            ./onmi ./gts/gt_xm_s${i}_trad.cnl ${file} &
            echo " "
        else
              echo "File ${file}"
              echo "Empty file. SKIPPED!"
        fi
#     # wait until all child processes are done
#     wait
    done
    echo " "
    # wait until all child processes are done
    wait
done

 
File OutputAnalysis/ococlus/Syn-1/run_1_res_ococlus_Syn-1_co.cnl is not empty. It has 7 lines.
 
 
# Average estimated membership in './gts/gt_xm_s1_trad.cnl': 0.6644 (3790 / 5704)
# Average estimated membership in 'OutputAnalysis/ococlus/Syn-1/run_1_res_ococlus_Syn-1_co.cnl': 0.6644 (3790 / 5704)
1


# OCoClus algorithm

This is the main algorithm of OCoClus.

In [2]:
def OCoClus(input_D,k=-1,e_obj=-1,e_att=-1):
    ### variable declaration
    if k == -1:
        k=sys.maxsize
    if e_obj == -1:
        e_obj = 1
    if e_att == -1:
        e_att = 1
    
    cost_model = sys.float_info.max # initial cost function of the model
    num_of_coclusters = 0
    final_coclusters = [] # store the attribute and objects clusters. final_coclusters[[C1_att,C1_obj],[Ck_att,Ck_obj]]
    pattern_model = [set(),set()]# Union between the found co-clusters [list of obj,list of att]
    cost_per_cocluster = []# stores the cost to build the cocluster
    history_cost_model = []
    ###
    
    D,N,data_dict,data_res_dict = get_data(input_D)
    for itertator in range(1,k+1):
        print("Iterator: ",itertator)
        if VERBOSE:
            print("Pattern model: ",pattern_model)
        C,E,new_cost_dense = find_dense_cocluster(data_dict, data_res_dict)
        if new_cost_dense >= 0:
            print("No relevant co-cluster can be found anymore.")
            break    
        C_expanded = expand_dense_cocluster(C, E, data_dict,e_obj,e_att)

        print("")
        new_cost = cost_function(len(pattern_model[0].union(C[1])),
                                              len(pattern_model[1].union(set(C[0]))))
        if VERBOSE:
            print("Dense co-cluster cost: ",new_cost_dense)
            print("Current model cost: ",cost_model)
            print("New model cost with the found co-cluster: ",new_cost)
            print("Attribute cluster:"+str(C[0])+", Object cluster: "+str(C[1]))
            print(" ")
        
        if new_cost < cost_model:
            final_coclusters.append(C)
            cost_model = new_cost
            num_of_coclusters += 1
            data_res_dict = update_residual_dataset(data_res_dict,C[0],C[1])
            pattern_model[0] = pattern_model[0].union(C[1])
            pattern_model[1] = pattern_model[1].union(set(C[0]))
        else:
            print("No co-cluster can be found anymore.")
            break
    
    print("Find global co-cluster is done.")
    if find_overlap:
        attributes_per_cluster = []
        objects_per_cluster = []
        for i in range(len(final_coclusters)):
            attributes_per_cluster.append(set(list(map(int,final_coclusters[i][0]))))
            objects_per_cluster.append(final_coclusters[i][1])

        print("\nFind overlapped co-clusters method")
        attribute_clusters, objects_clusters = findOverlap(attributes_per_cluster,objects_per_cluster)

        final_coclusters = []
        for i in range(len(attribute_clusters)):
            final_coclusters.append([attribute_clusters[i],objects_clusters[i]])

        print("\nNon-overlapped and overlapped co-clusters.")
        print("Number of co-clusters: ",len(final_coclusters))
        if VERBOSE:
            print("Final co-clusters: ",final_coclusters)
    else:
        print("\nNon-overlapped co-clusters.")
        print("Number of co-clusters: ",num_of_coclusters)
#         for i in range(len(final_coclusters)):
#             tmp_data = final_coclusters[i][1]
#             final_coclusters[i][1] = list(tmp_data)
        if VERBOSE:
            print("Final co-clusters: ",final_coclusters)

    return D,final_coclusters

### Find dense co-cluster

In [3]:
def find_dense_cocluster(input_dataset, residual_dataset):
    '''
    Input
        input_dataset: a dictionary with the list of objects per attribute. input_dataset[att] -> [objects_in_att]
        residual_dataset: it is a copy of input_dataset used to sort the attributes
    
    Output
        C: A dense co-cluster. It is a array with a list of attributes and a set of objects. C[att_list,obj_set]
    '''
    print("Find dense cocluster method.")
    att_dense_cocluster_list = []
#     att_dense_cocluster_set = set([])
    obj_dense_cocluster_list = []
    att_extension_list = []
    cost_dense_cocluster = sys.float_info.max
    
    sads = sort_att_ds(residual_dataset)
#     print("Sorted att: ",sads)
    cc_att = sads[0]
#     att_dense_cocluster_list.append(cc_att)
    att_dense_cocluster_list.append(cc_att)
    obj_dense_cocluter_set = set(residual_dataset[cc_att]) # get the objs forthe given attribute cc_att
    count_group_att = 1
#     cost_dense_cocluster = cost_function(len(pattern_model[0].union(obj_dense_cocluter_set)),
#                                          len(pattern_model[1].union(set(cc_att))))
    new_cost_function = cost_function(len(obj_dense_cocluter_set), count_group_att)
    if VERBOSE:
        print("New cost: "+str(new_cost_function)+", Cost dense: "+str(cost_dense_cocluster))
    cost_dense_cocluster = new_cost_function
    
    for next_att in range(1,len(sads)):
        cc_att_test = sads[next_att]
#         print("Attribute: ",cc_att_test)
        curr_cc_att = set(residual_dataset[cc_att_test])
        intersection_objs = obj_dense_cocluter_set.intersection(curr_cc_att)
#         print("Intersection test: ",intersection_objs)
        tmp = att_dense_cocluster_list.copy()
        tmp.append(cc_att_test)
#         new_cost_function = cost_function(len(pattern_model[0].union(intersection_objs)),
#                                           len(pattern_model[1].union(set(tmp))))
        new_cost_function = cost_function(len(intersection_objs),count_group_att+1)
#         print(intersection_objs,curr_cc_att)
#         print(len(intersection_objs),count_group_att+1)
        if VERBOSE:
            print("New cost: "+str(new_cost_function)+", Cost dense: "+str(cost_dense_cocluster))
            
        if  new_cost_function <= cost_dense_cocluster:
            att_dense_cocluster_list.append(cc_att_test)
            obj_dense_cocluter_set = intersection_objs
            cost_dense_cocluster = new_cost_function
            count_group_att += 1
        else:
            att_extension_list.append(cc_att_test)
    
    C = [att_dense_cocluster_list, obj_dense_cocluter_set]
    
    # no good rectangle was found
    if cost_dense_cocluster >= 0:
        att_extension_list = []
        C = [[],set()]
    else:
        C = [att_dense_cocluster_list, obj_dense_cocluter_set]
    
    return C, att_extension_list, cost_dense_cocluster

### Expand dense co-cluster

In [4]:
def expand_dense_cocluster(C,E,att_data_dict,e_obj,e_att):
    '''
    INPUT
        C = a tuple(list_atts, set_objs); list_atts has the list of attributes and set_objs is a set of objects
        E = list of attributes not present in C_a to be tested
        att_data_dict = a dict with list of objects per att; att_data_dict[att] -> [objects]
        e_obj = maximum object error 
        e_att = maximum attribute error 
    
    OUTPUT
        
    '''
    print("Expand co-cluster method.")
    added_att = True
    
    if not C[0]: # nothing good to discover
        pass
    else:
        curr_cost = cost_function(len(C[1]), len(C[0]),0,0)
        noise_added = 0 # the quantanty of noise added in the cluster during the process
        while(added_att):
            # try to add new objects to cocluster C
            try_new_objs = set(np.arange(0,len(D))).difference(C[1]) # get the objects not present in C
            if VERBOSE:
                print("# Try to extend the list of Objects #")
            for obj in try_new_objs:
                obj_quantanty = 0
                for att in C[0]:
                    if D[obj][int(att)] == 1:
                        obj_quantanty += 1
                if not_too_noisy(obj_quantanty, C, e_obj, e_att, att_data_dict, E, "obj"):
#                     print("Ruído valor: ",not_noise_val)
                    new_cost = cost_function(len(C[1])+1,len(C[0]),0,(noise_added+(len(C[0])-obj_quantanty)))
                    if new_cost <= curr_cost:
                        C[1].add(obj)
                        curr_cost = new_cost
                        noise_added += (len(C[0])-obj_quantanty)
                else:
                    if VERBOSE:
                        print("Object too noisy to be added. (Obj: "+str(obj)+")")

            added_att = False
            # try to add new attributes to cocluster C
            if VERBOSE:
                print("# Try to extend the list of Attributes #")
            
            while(len(E) != 0):
                att = E.pop(0)
                if VERBOSE:
                    print("ATT->"+str(att), end= " | ")
                att_obj_quantanty = len(C[1].intersection(set(att_data_dict[att])))
                if not_too_noisy(att_obj_quantanty, C, e_obj, e_att, att_data_dict, E, "att"):
                    new_cost = cost_function(len(C[1]),len(C[0])+1,0,(noise_added+(len(C[1])-att_obj_quantanty)))
                    if new_cost <= curr_cost:
                        C[0].append(str(att))
                        curr_cost = new_cost
                        added_att = True
                        noise_added += (len(C[1])-att_obj_quantanty)
                        break
                else:
                    if VERBOSE:
                        print("Attribute too noisy to be added. (Att: "+str(att)+")")

### Find overlapped co-clusters

In [5]:
def findOverlap(SetsC,SetsR):
    '''
    INPUT
        SetsC: It has K sets of attributes regarding each attribute cluster
        SetsR: It has K sets of objects regarding each object cluster
    
    OUTPUT
        columnClusters: A list with the attribute clusters
        rowClusters: A list with the object clusters
    '''
    newSetsColumns = []
    newSetsRows = []
    
    # merge sets that can overlap
    merge(SetsC,SetsR,newSetsColumns,newSetsRows)

    #Removing sets with redundant information
    removeSubsets(newSetsColumns,newSetsRows)
    
    columnClusters = newSetsColumns
    rowClusters = newSetsRows
    return columnClusters,rowClusters

# Support functions for the algorithms

In [6]:
def get_data(input_data):
    '''
    INPUT
        input_data: A panda dataframe of the input data file.
    
    OUTPUT
        Create global variables from the input data file to be used for the methods.
        D: A binary matrix from the input data.
        N: A noise binary matrix with the same size of D.
        data_dict: A dictionary to store D as a vertical representation.
        data_res_dict: A copy of data_dict used to sort the attributes of D and find unconvered elements.
        
    '''
    
    data_pd = input_data
    global data_dict
    data_dict = {}
    global data_res_dict
    data_res_dict = {}
    global D # input data as a binary matrix
    global N # noise matrix with the same size of D
    num_of_objects = 0
    num_of_attributes = 0
    
    max_val_att = 0
    for index, row in data_pd.iterrows():
        num_of_objects+=1
        object_data = row[0].split(" ")

        for attribute in object_data:
            if attribute != "":
                if int(attribute) > max_val_att:
                    max_val_att = int(attribute)
                if attribute in data_dict:
                    data_dict[attribute].append(index)
                else:
                    data_dict[attribute] = [index]
    data_res_dict = data_dict.copy()
#     num_of_attributes = len(data_dict)
    num_of_attributes = max_val_att+1
    
    D = np.zeros((num_of_objects,num_of_attributes),dtype=int)
    for key, values in data_dict.items():
        for line in values:
            D[line][int(key)] = 1        
    N = np.zeros((num_of_objects,num_of_attributes),dtype=int)
    return D,N,data_dict,data_res_dict

### Too noisy (line,col)?

In [7]:
def not_too_noisy(count_presence, C, e_obj, e_att, att_data_dict, E, dimension):
    num_of_atts = len(C[0])
    num_of_objs = len(C[1])
    if dimension == "obj":
        # obj must be present in at least (1-e_obj).||C_a||
        return count_presence >= ((1-e_obj) * num_of_atts) # return true if the obj is not too noisy
    else:
        # col must be present in at least (1-e_tt).||C_o||
        return count_presence >= ((1-e_att) * num_of_objs) # return true if the att is not too noisy

### cost function

In [8]:
def cost_function(numOfObj, numOfAtt, cov=0,noise=0):
    if VERBOSE:
        print("Num. objs: {0:2d}, Num. att: {1:2d}, Num. noise: {2:2d}".format(numOfObj,numOfAtt,noise))
    return ((numOfObj+numOfAtt) - (numOfObj*numOfAtt)) + cov + (2*noise)

In [9]:
def merge(SetsC, SetsR, newSetsColumns, newSetsRows):
    num_of_cluster = len(SetsC)
    
    # keep the original cluster
    for set_column_i in range(len(SetsC)):
        newSetsRows.append(SetsR[set_column_i])
        newSetsColumns.append(SetsC[set_column_i])
    
    revisit = True
    while(revisit):
        revisit = False
        tmp_r = []
        tmp_c = []
        size = len(newSetsColumns)
        for i in range(size-1):
            first_r = newSetsRows[i]
            first_c = newSetsColumns[i]
            for j in range(i+1,size):
                p_intersec_r = first_r.intersection(newSetsRows[j])
                p_intersec_c = first_c.union(newSetsColumns[j])
                if len(p_intersec_r) > 0:
                    tmp_r.append(p_intersec_r)
                    tmp_c.append(p_intersec_c)
        
        for i in range(len(tmp_c)):
            change = True
            for j in range(len(newSetsColumns)):
                if tmp_c[i].issubset(newSetsColumns[j]):
                    change = False
            if change:
                newSetsRows.append(tmp_r[i])
                newSetsColumns.append(tmp_c[i])
                revisit = True
    
    # start to find and merge all possible overlapping clusters
    for set_column_i in range(num_of_cluster):
        tested_pattern = SetsR[set_column_i]
        new_cols = SetsC[set_column_i]
        overlapped = False
        
        # finds with who the tested pattern overlaps
        set_overlap_id = []
        for i in range(num_of_cluster):
            if i != set_column_i:
                if len(tested_pattern.intersection(SetsR[i])) > 0:
                    set_overlap_id.append(i)    
        
        # Discover if exist overlaps between the sets in set_overlap_id
        sub_overlap_id = []
        for i in range(len(set_overlap_id)):
            try_combine_ids = [set_overlap_id[i]]
            added = False
            for j in range(i+1,len(set_overlap_id)):
                if len(SetsR[set_overlap_id[i]].intersection(SetsR[set_overlap_id[j]])) > 0:
                    try_combine_ids.append(set_overlap_id[j])
                    added = True
            sub_overlap_id.append(try_combine_ids)
        #Check if some pattern is isolated and was not added
        for id_pattern in set_overlap_id:
            added = False
            for combined_ids in sub_overlap_id:
                if id_pattern in combined_ids:
                    added = True
            if added == False:
                sub_overlap_id.append([id_pattern])
        
        # merge the analysed pattern with the overlapped combined patterns ids
        for pattern_ids in sub_overlap_id:
            tested_pattern = SetsR[set_column_i]
            new_cols = SetsC[set_column_i]
            for pattern_id in pattern_ids:
                tmp = tested_pattern.intersection(SetsR[pattern_id])
                tested_pattern = tmp
                new_cols = new_cols.union(SetsC[pattern_id])
            # save the new patterns
            newSetsColumns.append(new_cols)
            newSetsRows.append(tested_pattern)

In [10]:
def removeSubsets(setsColumns,setsRows):
    finalSetsCols = []
    finalSetsRows = []
    
    again = True
    # We are out of the loop when we do not have any subset to remove
    while(again):
        again = False
        for i in range(len(setsColumns)):
            isSubset = False
            currC = setsColumns[i]
            currR = setsRows[i]
            
            for j in range(len(setsColumns)):
                if i != j:
                    nextC = setsColumns[j]
                    nextR = setsRows[j]
                    if currC.issubset(nextC) and currR.issubset(nextR):
                        isSubset = True
        
            if isSubset or len(setsRows[i]) == 0:
                setsColumns.pop(i)
                setsRows.pop(i)
                again = True
                break
    
    #converting data type back to list
    for i in range(len(setsColumns)):
        setsColumns[i] = list(setsColumns[i])
        setsRows[i] = list(setsRows[i])

### Sort attributes in dataset

In [11]:
def sort_att_ds(data_res):
    freq_res_dict = {}
    for key,value in data_res.items():
        freq_res_dict[key] = len(value)
        
    # Create a list of tuples sorted by index 1 i.e. value field     
    listofTuples = sorted(freq_res_dict.items() , reverse=True, key=lambda x: x[1])
    # Iterate over the sorted sequence
    # for elem in listofTuples :
    #     print(elem[0] , " ::" , elem[1] )
#     print(listofTuples)
    sorted_attributes = [elem[0] for elem in listofTuples]
#     print("Sorted att: ",sorted_attributes)
    return sorted_attributes

### Update residual dataset

In [12]:
def update_residual_dataset(res_data, attributes_cocluster, objects_cocluster):
    for key, value in res_data.items():
        if key in attributes_cocluster:
            diff_objs = set(res_data[key]).difference(set(objects_cocluster))
            res_data[key] = list(diff_objs)
    return res_data

### Save results - check path

In [13]:
def check_path(path_method):
    current_dir = os.getcwd()
    print(current_dir)
    res = os.path.exists(path_method)
    # clean the folder to save new data
    if res:
        #check if it is empty
        dir_empty = os.listdir(path_method)
        if len(dir_empty) != 0:
    #         shutil.rmtree("OutputAnalysis/kmeans/")
            rm = !rm -r --preserve-root './OutputAnalysis/ococlus/'*
            if not rm:
                print("OCoClus' folder was cleaned.")
    #             os.chdir(path_method)
            else:
                print("sad")
                print(rm)
        else:
    #         print("Empty!")
            pass
    #         os.chdir(path_method)
    else: # nothing exist so create it
        # trying to insert to flase directory 
        try: 
    #         os.chdir(fd) 
            os.mkdir(path_method)
            print("The path was created: "+path_method)

        # Caching the exception     
        except: 
            print("Something wrong with specified directory. Exception- ", sys.exc_info())

### Save clustering result into a txt file

In [36]:
# def writeFileOutput(cols, rows, dataset, method='OCoClus', fileName='OCoClusResult'):
def writeFileOutput(co_clusters, dataset, method='OCoClus', fileName='OCoClusResult'):
    text = ""
#    for c in range(len(data.rows_)):
#        res = [i for i, val in enumerate(data.columns_[c]) if val]
#        for j in res:
#            text += str(j)+" "

#        res = [i for i, val in enumerate(data.rows_[c]) if val]
#        text += "["
#        for j in res:
#            text += str(j)+" "
#        text += "]\n"
    
    num_of_clusters = len(co_clusters)
    
#     for c in range(len(cols)):
    for c in range(num_of_clusters):
#         for i in cols[c]:
        for i in co_clusters[c][0]: # get the attributes in cluster c
            text += str(i)+" "
        
        text += "("+str(len(co_clusters[c][1]))+") [" # get the number of objects in clusters c
        for j in range(len(co_clusters[c][1])): # save in the file each obj
            if j+1 != len(co_clusters[c][1]):
                text += str(co_clusters[c][1][j])+" "
            else:
                text += str(co_clusters[c][1][j])
        text += "]\n"
    
    #print(text)
    if method == 'Dhillon':
        f = open('./datasets/outputs/'+fileName+'.txt', 'w+')#saving at dataset folder
        f.write(text)
        f.close()
        print("Output file saved in: "+"./datasets/outputs/"+fileName+".txt")
    elif method == 'Kluger':
        f = open('./datasets/outputs/'+fileName+'.txt', 'w+')#saving at dataset folder
        f.write(text)
        f.close()
        print("Output file saved in: "+"./datasets/outputs/"+fileName+".txt")
    elif method == 'OCoClus':
        f = open('./OutputAnalysis/ococlus/'+dataset+'/'+fileName+'.txt', 'w+')#saving at dataset folder
        f.write(text)
        f.close()
        print("Output file saved in: "+"./OutputAnalysis/ococlus/"+dataset+"/"+fileName+".txt")
    else:
        print("The output file was not generated. Method option not recognized.")

# Evaluation measure

### Reconstruction error

In [14]:
def Rec_error(data,clusters):
    '''
    This evaluation measure is computed during the algorithm life time.
    '''
    reconstructed_ococlus = np.zeros(data.shape,dtype=int)
    for nc in range(len(clusters)):
        for i in clusters[nc][1]: # object cluster
            for j in clusters[nc][0]: # attribute cluster
                reconstructed_ococlus[int(i)][int(j)] = 1
    print("Reconstruction error: ",np.sum(np.bitwise_xor(data,reconstructed_ococlus)))

### Omega format

In [15]:
def build_clustering_output_omega(co_clusters):
# def build_clustering_output_omega(rowClusters,columnClusters):
    '''
    Build the clustering output format to use in the omega index evaluation from Remy Cazabet version.
    It is optional and we just present this version as a complementary information. If you are interested,
    check it out on his team work group at https://github.com/isaranto/omega_index.
    '''
    
    num_of_clusters = len(co_clusters)    
    clustering = {}
    
    for nc in range(num_of_clusters):
        rowCluster = co_clusters[nc][1]
        columnCluster = co_clusters[nc][0]
        clustering["c"+str(nc)] = []
        
        for i in rowCluster:
            for j in columnCluster:
                clustering["c"+str(nc)].append(("01"+str(i)+"02"+str(j)))
        
    return clustering

### eXascale Infolab 
We used the xmeasure and OvpNMI project that pushished evaluation measures for overlapping task. We can check it on https://github.com/eXascaleInfolab/xmeasures or https://exascale.info/. Look their project on github to know how to use it.

In [16]:
def xmeasures_format(dict_gt):
    '''
    This function build the xmeasure format to use it on their evaluation measure.
    '''
    newData = []
    for i in range(len(dict_gt)):
#         print(dict_gt['c'+str(i)])
        stringLine = dict_gt['c'+str(i)][0]
        for j in range(1,len(dict_gt['c'+str(i)])):
#             stringLine = stringLine+" "+dict_gt['c'+str(i)][j]
            stringLine += " "+dict_gt['c'+str(i)][j]
        newData.append(stringLine)
    
    return newData